In [71]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale,Normalizer,OneHotEncoder
from sklearn.preprocessing import normalize
from keras.models import Sequential,load_model
from tensorflow.python.keras.saving.saved_model import load as saved_model_load
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import keras.optimizers 
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [72]:
train = pd.read_csv('task4.training.csv')

In [73]:
train2 = pd.read_csv('task4.training.csv')

In [74]:
test = pd.read_csv('task4.test.WITHOUT.csv')

In [75]:
train_answer = train['class']

In [76]:
train['class']

0       F
1       E
2       G
3       H
4       C
       ..
9995    G
9996    A
9997    I
9998    B
9999    C
Name: class, Length: 10000, dtype: object

In [77]:
train_last = pd.get_dummies(train['f24'])
test_last = pd.get_dummies(test['f24'])

In [78]:
from sklearn.decomposition import FastICA 
ICA = FastICA(n_components=1, random_state=12) 
train_last=ICA.fit_transform(train_last)
test_last = ICA.fit_transform(test_last)

In [79]:
train = train.drop(columns = 'class')
train = train.drop(columns = 'f24')
test = test.drop(columns = 'f24')

In [80]:
train = np.array(train,dtype=np.float32)
test = np.array(test,dtype=np.float32)

In [81]:
meanlist=[]
stdlist =[]

In [13]:
sc = StandardScaler()
sc.fit(train)
train = sc.transform(train)
test = sc.transform(test)

In [14]:
# for i in range(np.shape(train)[1]):
#     tmp = np.sum(train[:,i])
#     mean = tmp/np.shape(train)[0]
#     meanlist.append(mean)
#     std = np.std(train[:,i])
#     stdlist.append(std)
#     for j in range(np.shape(train)[0]):
#         train[j][i] = (train[j][i]-mean)/std

In [15]:
# meanlist

In [16]:
# stdlist

In [17]:
# for i in range(np.shape(test)[1]):
#     mean = meanlist[i]
#     std  = stdlist[i]
#     for j in range(np.shape(test)[0]):
#         test[j][i] = (test[j][i]-mean)/std

In [18]:
# normalizer = Normalizer().fit(train) 
# train = normalizer.transform(train)

In [82]:
train = np.append(train,train_last,axis=1)
test = np.append(test,test_last,axis=1)

In [83]:
np.shape(train)

(10000, 25)

In [84]:
ans = pd.get_dummies(train_answer)

In [85]:
ans = np.array(ans)

In [86]:
train_feature_len = np.shape(train)[1]

In [87]:
train_x,val_x,train_y,val_y = train_test_split(train,ans,test_size=0.3,random_state=44)

## Decision Tree (0.10100)

In [24]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)

In [25]:
decisiontree_ans =  clf.predict(val_x)

In [26]:
f1_score(decisiontree_ans,val_y,average='weighted')

0.09842952237011923

In [27]:
# alph = ['A','B','C','D','E','F','G','H','I','J']
# prediction_final=[]
# for i in range(np.shape(decisiontree_ans)[0]):
#     index = np.argmax(decisiontree_ans[i], axis=0)
#     prediction_final.append(alph[index])

In [28]:
# tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])
# tt = pd.DataFrame(tt)
# tt.to_csv('test_decision_tree.csv',index=False)

## KNN (0.01919)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(train_x, train_y)
knn_ans = knn.predict(val_x)

In [30]:
f1_score(knn_ans,val_y,average='weighted')

0.019195528086035685

## SVM(0.1044)

In [31]:
train_x2,val_x2,train_y2,val_y2 = train_test_split(train,train2['class'].values,test_size=0.3,random_state=44)

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_x2, train_y2 )
svm_ans = clf.predict(val_x)

In [33]:
f1_score(svm_ans,val_y2,average='weighted')

0.10446160425910936

## Weka J48

In [35]:
from weka.classifiers import Classifier


AssertionError: Weka JAR file /usr/share/java/weka.jar not found. Ensure the file is installed or update your environment's WEKA_JAR_PATH to only include valid locations.

In [25]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [88]:
lr= 1e-3
opti =keras.optimizers.Adam(learning_rate=lr)
model3 = Sequential()  
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(10, activation = 'softmax',use_bias=True))
para3 = model3.compile(optimizer = opti, loss = 'categorical_crossentropy', metrics = ['acc',f1_m,precision_m, recall_m])


In [89]:
# model3=load_model('ML_task2.h5')

In [90]:
# checkpoint
filepath="weights_with_normalize.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
callbacks_list = [checkpoint]

In [91]:
#model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [92]:
# print('Training ------------')
# model3.fit(train_x, train_y, validation_split=0.33, epochs=1500,batch_size=128,callbacks=callbacks_list,verbose=0) #2000

In [106]:
print('Training ------------')
model3.fit(train_x, train_y, epochs=10,batch_size=128) #2000

Training ------------
Epoch 1/10
55/55 [==============================] - 0s 1ms/step - loss: 1.3922 - acc: 0.4916 - f1_m: 0.4229 - precision_m: 0.7573 - recall_m: 0.2946
Epoch 2/10
55/55 [==============================] - 0s 2ms/step - loss: 1.3878 - acc: 0.4891 - f1_m: 0.4280 - precision_m: 0.7644 - recall_m: 0.2980
Epoch 3/10
55/55 [==============================] - 0s 1ms/step - loss: 1.3961 - acc: 0.4916 - f1_m: 0.4232 - precision_m: 0.7549 - recall_m: 0.2951
Epoch 4/10
55/55 [==============================] - 0s 1ms/step - loss: 1.4054 - acc: 0.4886 - f1_m: 0.4216 - precision_m: 0.7550 - recall_m: 0.2932
Epoch 5/10
55/55 [==============================] - 0s 1ms/step - loss: 1.4122 - acc: 0.4871 - f1_m: 0.4191 - precision_m: 0.7458 - recall_m: 0.2924
Epoch 6/10
55/55 [==============================] - 0s 3ms/step - loss: 1.4181 - acc: 0.4811 - f1_m: 0.4135 - precision_m: 0.7342 - recall_m: 0.2886
Epoch 7/10
55/55 [==============================] - 0s 2ms/step - loss: 1.4125 - acc

In [107]:
# model3.save('ML_task2.h5')

In [108]:
# model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [109]:
print('\nTesting ------------')
loss, accuracy, f1_score, precision, recall = model3.evaluate(val_x, val_y, verbose=0)
print('\ntest loss: ', loss)
print('\ntest f1_score: ', f1_score) #with normal best 0.02490120939910412


Testing ------------

test loss:  6.128544807434082

test f1_score:  0.059657808393239975


In [252]:
test.shape

(2000, 26)

In [253]:
np.sum(test[:,1])

-67.57402353528985

In [258]:
prediction = model3.predict(test)

In [259]:
np.argmax(prediction[0], axis=0)

6

In [260]:
alph = ['A','B','C','D','E','F','G','H','I','J']

In [261]:
prediction_final=[]

In [262]:
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [43]:
# prediction_final

In [264]:
tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])

In [265]:
tt = pd.DataFrame(tt)

In [266]:
tt.to_csv('test.csv',index=False)